In [1]:
path = './zomato.csv'

In [2]:
import csv
import os
import glob
import numpy as np
import math
#scikit learn
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def cleaning(sentence):
    sentence = sentence.replace('\ufeff','')
    sentence = sentence.replace('\n',' ')
    sentence = sentence.replace('\r',' ')
    sentence = sentence.replace('\n\n',' ')
    sentence = sentence.replace('\r\r',' ')
    sentence = sentence.replace('.',' ')
    sentence = sentence.replace(',',' ')
    sentence = sentence.replace('"',' ')
    sentence = sentence.replace('!',' ')
    sentence = sentence.replace('?',' ')
    sentence = sentence.replace('/',' ')
    sentence = sentence.replace('-',' ')
    sentence = sentence.lstrip()
    return sentence.lower()

In [4]:
def convert(sentences):
    import re
    cleanEmoticon = []
    for sentence in sentences:
        try:
            highpoints = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
        except re.error:
            highpoints = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')
    
        cleanEmoticon.append(highpoints.sub(u'\u25FD', sentence))
    return cleanEmoticon

In [5]:
x = []
y = []
with open(path) as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])

In [6]:
randomize = np.arange(len(x))
np.random.shuffle(randomize)

In [7]:
x_train = x[:85]
y_train = y[:85]

x_test = x[85:]
y_test = y[85:]

In [8]:
def preprocess(sentences): 
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    result =[]
    for sentence in sentences:
        result.append(stemmer.stem(stopword.remove(sentence)))
    return result

In [9]:
# stem_test = ['dia biasanya makan nasi padang', 
#              'aku menyukainya dengan tulus dan raisa', 
#              'ku tak kan pernah memilikinya sampai kapanpun', 
#              'karena dia sudah memilikinya & dimilikinya']

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [10]:
x_train = convert(x_train)
x_test = convert(x_test)

### Scikit-Learn Tweaks
Reference:
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [11]:
from sklearn import svm

text_clf = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', svm.SVC(kernel='linear')),
])
text_clf = text_clf.fit(x_train, y_train)

/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
accuracy_score(y_test, y_pred)

0.7333333333333333

In [14]:
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(x_train, y_train)

In [15]:
y_pred = text_clf.predict(x_test)

In [16]:
accuracy_score(y_test, y_pred) #persebaran data dalam 1 kelas tidak seimbang.

0.8

In [17]:
s_l = classification_report(y_test, y_pred,target_names=['0','1'])

In [18]:
print(s_l)

             precision    recall  f1-score   support

          0       1.00      0.77      0.87        13
          1       0.40      1.00      0.57         2

avg / total       0.92      0.80      0.83        15



In [19]:
confusion_matrix(y_test, y_pred)

array([[10,  3],
       [ 0,  2]])

In [20]:
from sklearn.model_selection import KFold

x = np.concatenate((x_train,x_test))
y = np.concatenate((y_train,y_test))
        
randomize = np.arange(len(x))
np.random.shuffle(randomize)

acc = []
folds = 6
kf = KFold(n_splits=folds)
for train, test in kf.split(x):
    x_train, x_test, y_train, y_test = x[train],x[test],y[train],y[test]
    
    text_clf = Pipeline([('vect', CountVectorizer()),
                     ('clf', MultinomialNB()),
    ])
    text_clf = text_clf.fit(x_train, y_train)
    y_pred = text_clf.predict(x_test)
    acc.append(accuracy_score(y_test, y_pred))
print("Mean Acc of",folds,"Fold:",np.mean(acc))

Mean Acc of 6 Fold: 0.7107843137254902
